In [1]:
# Before running this code in Google Colab, upload the following files:
# 1. best.pt: Weights of the fine-tuned YOLOv8n model trained on the MOT17.
# 2. Sample.mp4: A trimmed sample video from the MOT17 dataset.
# 3. SFSORT.py: The tracking algorithm file.

# After running the code, an output video named 'output.mp4' will be generated.
# This video is a processed version of 'Sample.mp4' and displays bounding boxes
# and unique IDs for each detected object (people).

In [2]:
# Install lapx, required for SFSORT
!pip install lapx

#Install ultralytics to access the object detector
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 878.7/878.7 kB 27.4 MB/s eta 0:00:00


In [1]:
# Import essential libraries
import numpy as np
import cv2

from ultralytics import YOLO
from ultralytics.utils.torch_utils import select_device
from random import randrange

from SFSORT import SFSORT

In [4]:
import torch

model = YOLO('best.pt', 'detect')


In [9]:
model = model.export(format = 'onnx')

Ultralytics 8.3.78 🚀 Python-3.10.12 torch-2.6.0+cu124 CPU (Intel Core(TM) i5-10210U 1.60GHz)

PyTorch: starting from 'best.pt' with input shape (1, 3, 1440, 1440) BCHW and output shape(s) (1, 5, 42525) (6.1 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.9/142.9 KB 17.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 7.2s, installed 2 packages: ['onnx>=1.12.0', 'onnxslim']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.48...
ONNX: export success ✅ 8.7s, saved as 'best.onnx' (12.3 MB)

Export complete (10.0s)
Results saved to /home/yuichi-katagiri/[MAIN]/OUTDOOR/Digital-Department-Project/MAIN/SFSORT
Predict:         yolo predict task=detect model=best.onnx imgsz=1440  
Validate:   

In [6]:
import torch.onnx 

#Function to Convert to ONNX 
def Convert_ONNX(): 

    # set the model to inference mode 
    model.eval() 

    # Let's create a dummy input tensor  
    dummy_input = torch.rand(1, 3, 640, 640,  requires_grad=True)  

    # Export the model   
    torch.onnx.export(model,         # model being run 
         dummy_input,       # model input (or a tuple for multiple inputs) 
         "ImageClassifier.onnx",       # where to save the model  
         export_params=True,  # store the trained parameter weights inside the model file 
         opset_version=11,    # the ONNX version to export the model to 
         do_constant_folding=True) 
    print(" ") 
    print('Model has been converted to ONNX')

In [ ]:
something = model.predict(torch.rand(1, 3, 640, 640))


WARNING ⚠️ torch.Tensor inputs should be BCHW i.e. shape(1, 3, 640, 640) divisible by stride 32. Input shape(3, 800, 1440) is incompatible.
0: 800x1440 (no detections), 394.8ms
Speed: 1.6ms preprocess, 394.8ms inference, 12.8ms postprocess per image at shape (1, 3, 800, 1440)


In [7]:
Convert_ONNX()

New https://pypi.org/project/ultralytics/8.3.79 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.78 🚀 Python-3.10.12 torch-2.6.0+cu124 CPU (Intel Core(TM) i5-10210U 1.60GHz)
engine/trainer: task=detect, mode=train, model=best.pt, data=/content/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=1440, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frame

RuntimeError: Dataset '/content/data.yaml' error ❌ '/content/data.yaml' does not exist

In [4]:
# Instantiate an object detector
# To use YOLOv8n without fine-tuning, replace 'best.pt' with 'yolov8n.pt'
model = YOLO('best.pt', 'detect')

# Check for GPU availability
device = select_device('0')
# Devolve the processing to selected devices
model.to(device)

Ultralytics 8.3.25 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [5]:
# Load the video file
cap = cv2.VideoCapture('/content/Sample.mp4')

# Get the frame rate, frame width, and frame height
frame_rate = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the MP4 codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 30.0, (frame_width, frame_height))

# Organize tracker arguments into standard format
tracker_arguments = {"dynamic_tuning": True, "cth": 0.5,
                      "high_th": 0.6, "high_th_m": 0.1,
                      "match_th_first": 0.67, "match_th_first_m": 0.05,
                      "match_th_second": 0.2, "low_th": 0.1,
                      "new_track_th": 0.7, "new_track_th_m": 0.1,
                      "marginal_timeout": (7 * frame_rate // 10),
                      "central_timeout": frame_rate,
                      "horizontal_margin": frame_width // 10,
                      "vertical_margin": frame_height // 10,
                      "frame_width": frame_width,
                      "frame_height": frame_height}

# Instantiate a tracker
tracker = SFSORT(tracker_arguments)

# Define a color list for track visualization
colors = {}

# Process each frame of the video
while cap.isOpened():
   # Load the frame
  ret, frame = cap.read()
  if not ret:
      break

  # Detect people in the frame
  prediction = model.predict(frame, imgsz=(800,1440), conf=0.1, iou=0.45,
                              half=False, device=device, max_det=99, classes=0,
                              verbose=False)

  # Exclude additional information from the predictions
  prediction_results = prediction[0].boxes.cpu().numpy()

  # Update the tracker with the latest detections
  tracks = tracker.update(prediction_results.xyxy, prediction_results.conf)

  # Skip additional analysis if the tracker is not currently tracking anyone
  if len(tracks) == 0:
      continue

  # Extract tracking data from the tracker
  bbox_list = tracks[:, 0]
  track_id_list = tracks[:, 1]

  # Visualize tracks
  for idx, (track_id, bbox) in enumerate(zip(track_id_list, bbox_list)):
    # Define a new color for newly detected tracks
    if track_id not in colors:
        colors[track_id] = (randrange(255), randrange(255), randrange(255))

    color = colors[track_id]

    # Extract the bounding box coordinates
    x0, y0, x1, y1 = map(int, bbox)

    # Draw the bounding boxes on the frame
    annotated_frame = cv2.rectangle(frame, (x0, y0), (x1, y1), color, 2)
    # Put the track label on the frame alongside the bounding box
    cv2.putText(annotated_frame, str(track_id), (x0, y0-5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)


  # Write the frame to the output video file
  out.write(annotated_frame)

# Release everything when done
cap.release()
out.release()